In [4]:
using Pkg
Pkg.activate(joinpath(Pkg.devdir(), "TaskGraphs"))

"/home/kylebrown/.julia/dev/TaskGraphs/Project.toml"

In [6]:
using LightGraphs, MetaGraphs, GraphUtils
using LinearAlgebra
using TaskGraphs
using Gurobi
using JuMP, MathOptInterface
using TOML

In [47]:
i = 1
project_spec, problem_spec, robot_ICs, assignments, env_graph = initialize_toy_problem_9(;
        verbose=false,Δt_op=0,Δt_collect=[i,0],Δt_deliver=[0,0]
        );
model = formulate_JuMP_optimization_problem(problem_spec,Gurobi.Optimizer;cost_model=:SumOfMakeSpans)
optimize!(model)
optimal = (termination_status(model) == MathOptInterface.OPTIMAL)
optimal_TA_cost = Int(round(value(objective_function(model))));
@show optimal
@show optimal_TA_cost

Academic license - for non-commercial use only
(j, j2) = (1, 2)
(j, j2, s0[j], s0[j2]) = (1, 2, 5, 5)
Academic license - for non-commercial use only
optimal = true
optimal_TA_cost = 8


8

In [48]:
to0 = Int.(round.(value.(model[:to0])))
tor = Int.(round.(value.(model[:tor])))
toc = Int.(round.(value.(model[:toc])))
tod = Int.(round.(value.(model[:tod])))
tof = Int.(round.(value.(model[:tof])))
@show Int.(round.(value.(model[:x])))
@show to0
@show tor
@show toc
@show tod
@show tof;

Int.(round.(value.(model[:x]))) = [0 1; 1 0; 0 0; 0 0]
to0 = [0, 0]
tor = [2, 1]
toc = [3, 1]
tod = [6, 2]
tof = [6, 2]


In [43]:
function TaskGraphs.formulate_JuMP_optimization_problem(G,Drs,Dss,Δt,Δt_collect,Δt_deliver,to0_,tr0_,root_nodes,weights,s0,sF,optimizer;
    TimeLimit=100,
    OutputFlag=0,
    cost_model=:MakeSpan
    )

    model = Model(with_optimizer(optimizer,
        TimeLimit=TimeLimit,
        OutputFlag=OutputFlag
        ))
    M = size(Dss,1)
    N = size(Drs,1)-M
    @variable(model, to0[1:M] >= 0.0) # object availability time
    @variable(model, tor[1:M] >= 0.0) # object robot arrival time
    @variable(model, toc[1:M] >= 0.0) # object collection complete time
    @variable(model, tod[1:M] >= 0.0) # object deliver begin time
    @variable(model, tof[1:M] >= 0.0) # object termination time
    @variable(model, tr0[1:N+M] >= 0.0) # robot availability time

    # Assignment matrix x
    @variable(model, x[1:N+M,1:M], binary = true) # x[i,j] ∈ {0,1}
    @constraint(model, x * ones(M) .<= 1)         # each robot may have no more than 1 task
    @constraint(model, x' * ones(N+M) .== 1)      # each task must have exactly 1 assignment
    for (i,t) in tr0_
        # start time for robot i
        @constraint(model, tr0[i] == t)
    end
    for (j,t) in to0_
        # start time for task j (applies only to tasks with no prereqs)
        @constraint(model, to0[j] == t)
    end
    # constraints
    Mm = Matrix{Float64}(I,N+M,N+M) * (sum(Drs) + sum(Dss)) # for big-M constraints
    MMm = typemax(Int) # sum(Drs) + sum(Dss) # for scalar big-M constraints in station ordering
    for j in 1:M
        # constraint on task start time
        if !is_leaf_node(G,j)
            for v in inneighbors(G,j)
                @constraint(model, to0[j] >= tof[v] + Δt[j])
            end
        end
        # constraint on dummy robot start time (corresponds to moment of object delivery)
        @constraint(model, tr0[j+N] == tof[j])
        # dummy robots can't do upstream jobs
        upstream_jobs = [j, map(e->e.dst,collect(edges(bfs_tree(G,j;dir=:in))))...]
        for v in upstream_jobs
            @constraint(model, x[j+N,v] == 0)
        end
        # lower bound on task completion time (task can't start until it's available).
        # tof[j] = to0[j] + Dss[j,j] + slack[j]
        @constraint(model, tor[j] >= to0[j])
#         @constraint(model, tof[j] >= tor[j] + Dss[j,j] + Δt_collect[j] + Δt_deliver[j])
        # bound on task completion time (assigned robot must first complete delivery)
        # Big M constraint (thanks Oriana!): When x[i,j] == 1, this constrains the final time
        # to be no less than the time it takes for the delivery to be completed by robot i.
        # When x[i,j] == 0, this constrains the final time to be greater than a large negative
        # number (meaning that this is a trivial constraint)        
        @constraint(model, tor[j] .- (tr0 + Drs[:,j]) .>= -Mm*(1 .- x[:,j]))
        @constraint(model, toc[j] == tor[j] + Δt_collect[j])
        @constraint(model, tod[j] == toc[j] + Dss[j,j])
        @constraint(model, tof[j] == tod[j] + Δt_deliver[j])
#         @constraint(model, tof[j] >= tor[j] + Dss[j,j] + Δt_collect[j] + Δt_deliver[j])
        # "Job-shop" constraints specifying that no station may be double-booked. A station
        # can only support a single COLLECT or DEPOSIT operation at a time, meaning that all
        # the windows for these operations cannot overlap. In the constraints below, t1 and t2
        # represent the intervals for the COLLECT or DEPOSIT operations of tasks j and j2, 
        # respectively. If eny of the operations for these two tasks require use of the same 
        # station, we introduce a 2D binary variable y. if y = [1,0], the operation for task 
        # j must occur before the operation for task j2. The opposite is true for y == [0,1]. 
        # We use the big M method here as well to tightly enforce the binary constraints.
        for j2 in j+1:M
            if (s0[j] == s0[j2]) || (s0[j] == sF[j2]) || (sF[j] == s0[j2]) || (sF[j] == sF[j2])
                @show j, j2
                if s0[j] == s0[j2]
                    @show j, j2, s0[j], s0[j2]
                    t1 = [tor[j], toc[j]]
                    t2 = [tor[j2], toc[j2]]
                elseif s0[j] == sF[j2]
                    @show j, j2, s0[j], sF[j2]
                    t1 = [tor[j], toc[j]]
                    t2 = [tod[j2], tof[j2]]
                elseif sF[j] == s0[j2]
                    @show j, j2, sF[j], s0[j2]
                    t1 = [tod[j], tof[j]]
                    t2 = [tor[j2], toc[j2]]
                elseif sF[j] == sF[j2]
                    @show j, j2, sF[j], sF[j2]
                    t1 = [tod, tof[j]]
                    t2 = [tod, tof[j2]]
                end
                tmax = @variable(model)
                tmin = @variable(model)
                y = @variable(model, binary=true)
                @constraint(model, tmax >= t1[1])
                @constraint(model, tmax >= t2[1])
                @constraint(model, tmin <= t1[2])
                @constraint(model, tmin <= t2[2])
                
                @constraint(model, tmax - t2[1] <= (1 - y)*MMm)
                @constraint(model, tmax - t1[1] <= y*MMm)
                @constraint(model, tmin - t1[2] >= (1 - y)*-MMm)
                @constraint(model, tmin - t2[2] >= y*-MMm)
#                 @constraint(model, t1[2] - tmin <= (1 - y)*MMm)
#                 @constraint(model, t2[2] - tmin <= y*MMm)
                @constraint(model, tmin + 1 <= tmax)
            end
        end
    end
    # cost depends only on root node(s)
    # @objective(model, Min, tof[M])
    if cost_model == :SumOfMakeSpans
        @objective(model, Min, sum(map(v->tof[v]*get(weights,v,0.0), root_nodes)))
    elseif cost_model == :MakeSpan
        @variable(model, T)
        @constraint(model, T .>= tof)
        @objective(model, Min, T)
    end
    model;
end